In [1]:
import shutil
from subprocess import check_output
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import os
#import matplotlib.pyplot as plt
import pandas as pd
#from shapely.geometry import Polygon,
import RClib as RClib
import objectdetectionlib as ODlib
import json



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
True
11.3


In [2]:
%load_ext autoreload
%autoreload 2

config = RClib.loadconfigs('.\config_scanner.json')

if Path(config['workspace'] + '/rcprocessingdf.pkl').is_file() and config['use_initialdf'] :
    print('Read scandf from pickle')
    scandf = pd.read_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))
else:
    scandf = RClib.provide_scandf(inputdirectory=config['workspace'], imageformat = config['baseimageformat'])
    if config['use_startpointtransformation']:
        scandf = scandf.apply(RClib.createGCPfile_forsedimentcores, axis=1)
    scandf= scandf.apply(RClib.defineRawTherapeeOutput, foldername='02_TIF', axis=1)
    scandf= scandf.apply(RClib.defineRealityCaptureOutput, foldername='03_RC', axis=1)
    #scandf.to_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))

for index, scan in scandf.iterrows():
    #RClib.visualize_rcbox(scan['rcbox'])
    print('Processing scan: ' + scan['id'])
    print('Number of images: ' + str(len(scan['imagedf'])))
    #print('Number of masks: ' + str(len(scan['imagedf'][scan['imagedf']['maskimg_path'].notnull()])))


Processing scan: camrun-20210628_074128-wy193
Number of images: 1808
Processing scan: camrun-20210628_074336-wy194
Number of images: 932


In [3]:
print(scan['imagedf'])


                                           rawimg_path
0    E:\testSAM\test_alignqual\subtest\camrun-20210...
1    E:\testSAM\test_alignqual\subtest\camrun-20210...
2    E:\testSAM\test_alignqual\subtest\camrun-20210...
3    E:\testSAM\test_alignqual\subtest\camrun-20210...
4    E:\testSAM\test_alignqual\subtest\camrun-20210...
..                                                 ...
927  E:\testSAM\test_alignqual\subtest\camrun-20210...
928  E:\testSAM\test_alignqual\subtest\camrun-20210...
929  E:\testSAM\test_alignqual\subtest\camrun-20210...
930  E:\testSAM\test_alignqual\subtest\camrun-20210...
931  E:\testSAM\test_alignqual\subtest\camrun-20210...

[932 rows x 1 columns]


In [4]:


scandf2=pd.DataFrame()

for index,scan in scandf.iterrows():
    if not scan['imagedf'] is None and config['developimages']:
        if 'pp3file' in scan.keys() and scan['pp3file']:    
            scan['imagedf'] = scan['imagedf'].apply(RClib.developwithRawTherapee, pp3filepath=scan['pp3file'][0],  outputfolderpath=scan['RToutputfolder'], axis=1)     
            scandf2 = scandf2.append(scan)

    else:
        #scan['imagedf'] = scan['imagedf'].apply(RClib.baseimageIsDevimage, axis=1)
        print('Number of images after development: ' + str(len(scan['imagedf'])))
        scandf2 = scandf2.append(scan)

Number of images after development: 1808
Number of images after development: 932


C:\Users\mhaibt\AppData\Local\Temp\ipykernel_17520\2191258122.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf2 = scandf2.append(scan)
C:\Users\mhaibt\AppData\Local\Temp\ipykernel_17520\2191258122.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf2 = scandf2.append(scan)


In [5]:
%load_ext autoreload
%autoreload 2

scandf2a = pd.DataFrame()
for index,scan in scandf2.iterrows():
    #print('Number of images before masking: ' + str(len(scan['imagedf'])))
    #print('Start masking scan: ' + scan['id'])
    if not scan['imagedf'] is None and config['maskobject_QS3D']:
        scan['imagedf'] = scan['imagedf'].apply(RClib.provide_imageinfo_scanner, axis=1)
        #print('Number of images after imageinfo: ' + str(len(scan['imagedf'])))
        if 'colorreference_path' in config.keys() and config['colorreference_path']:
            print('Color correction with reference: ' + config['colorreference_path'])
            # load color reference
            with open(config['colorreference_path'], 'r') as f:
                colorreference = json.load(f)

            scan['imagedf'] = scan['imagedf'].apply(ODlib.colorcorrection, axis=1, colorreference=colorreference, outputfolderpath=scan['RToutputfolder'])
        print('dev img path: ' + scan['dev-img_path'])
        scan['imagedf'] = scan['imagedf'].apply(ODlib.maskoutobject_QS3D, axis=1)
        #print('Number of images before shotnumber: ' + str(len(scan['imagedf'])))
        scan['imagedf'] = RClib.add_shotnumber(scan['imagedf'] )

        
    else:
        print('no images to mask out')
    #print('Number of images after masking: ' + str(len(scan['imagedf'])))
    scandf2a = scandf2a.append(scan)

    #scandf2a = RClib.add_shotnumber(scandf2a)
#print(scandf2a)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Color correction with reference: E:/testSAM/RCutils/colorcalib_whiteblack_config.json


KeyError: 'dev-img_path'

In [ ]:
%load_ext autoreload
%autoreload 2
### For dev purpose the purpose will align the images of the first shot from each object first
        
scandf3=pd.DataFrame()
scandfBefore1shot=scandf2a.copy()
if config['align_1shot']:
    for index,scan in scandf2a.iterrows():
        if not scan['imagedf'] is None :
  
            pseudoscan = scan
            print(scan['imagedf']['roundnumber'])
            imagedf_only1shot = scan['imagedf'][scan['imagedf']['shotnumber']==1]
            pseudoscan['imagedf']  = imagedf_only1shot[imagedf_only1shot['roundnumber']=='round0']
            pseudoscan = RClib.makeImagelist(pseudoscan, imagelistname=scan['id'] +'_1shot')  
            pseudoscan = RClib.makeImagelist(pseudoscan, imagelistname= scan['id'] +'_masks' , imagefield='maskimg_path')
            pseudoscan = RClib.createRCproject(pseudoscan)
            pseudoscan =RClib.readProcessingstateFile(pseudoscan)    
            if not RClib.checkProcessingstate(pseudoscan,command='commandlist_alignsettings'):
                pseudoscan['commandlist_alignsettings']=Path(config['alignmentsettings'])
                RClib.executeRCCMDuseRCproject(pseudoscan, rccmdpathfield='commandlist_alignsettings' )
            if not RClib.checkProcessingstate(pseudoscan,command='align_1shot-rccmdpath'):
                pseudoscan['commandlist_align_1shot'] = []
                pseudoscan['commandlist_align_1shot'].append('-add ' + str(scan['list_dev-img_path']))
                if config['maskobject_QS3D']:
                    pseudoscan['commandlist_align_1shot'].append('-add ' + str(scan['list_maskimg_path']))
                if 'imageparams' in config and config['imageparams'] == 'constant':
                    pseudoscan['commandlist_align_1shot'].append('-selectAllImages')
                    pseudoscan['commandlist_align_1shot'].append('-setFeatureSource 2')
                    pseudoscan['commandlist_align_1shot'].append('-setPriorLensGroup 1')
                    pseudoscan['commandlist_align_1shot'].append('-setPriorCalibrationGroup 1')
                if 'imageparams' in config and config['imageparams'] == 'variable':
                    pseudoscan['commandlist_align_1shot'].append('-selectAllImages')
                    pseudoscan['commandlist_align_1shot'].append('-setFeatureSource 2')
                    pseudoscan['commandlist_align_1shot'].append('-setPriorLensGroup -1')
                    pseudoscan['commandlist_align_1shot'].append('-setPriorCalibrationGroup -1')
                if 'imageparams' in config and config['imageparams'] == 'fixed' and Path(config['imageparamsfile']).is_file():
                    #read imageparams from json-file which is defined in config
                    imageparams = pd.read_json(config['imageparamsfile'], orient='records')
                    #makeImagelist of each group of images with same 'cam_id'
                    # and add to commandlist_align
                    for name, group in pseudoscan['imagedf'].groupby('cam_id'):
                        pseudopseudoscan = pseudoscan
                        pseudopseudoscan['imagedf'] = group
                        group = RClib.makeImagelist(pseudopseudoscan, imagelistname=pseudoscan['id'] + '_' + str(name) + '_images' , imagefield='dev-img_path')
                        pseudoscan['commandlist_align_1shot'].append('-importImageSelection ' + str(group['list_dev-img_path']))
                        pseudoscan['commandlist_align_1shot'].append('-editInputSelection "inpCalibration=2"')
                        pseudoscan['commandlist_align_1shot'].append('-editInputSelection "inpFocal=' + str(imageparams[imageparams['cam_id']== name].iloc[0]['f_mean'])+ '"')
                        pseudoscan['commandlist_align_1shot'].append('-editInputSelection "inpPPX=' + str(imageparams[imageparams['cam_id']== name].iloc[0]['px_mean'])+ '"')
                        pseudoscan['commandlist_align_1shot'].append('-editInputSelection "inpPPY=' + str(imageparams[imageparams['cam_id']== name].iloc[0]['py_mean'])+ '"')
                        pseudoscan['commandlist_align_1shot'].append('-editInputSelection "inpDistortion=2"') 
                        pseudoscan['commandlist_align_1shot'].append('-editInputSelection "inpDistortionModel=1"')
                        pseudoscan['commandlist_align_1shot'].append('-editInputSelection "inpRadial1=-0.049831"')
                if config['use_gcps_1shot']:
                    pseudoscan['commandlist_align_1shot'].append('-detectMarkers')
                    if Path(config['constantgcps']).is_file() and Path(config['gcpsparamfile']).is_file():
                        pseudoscan['commandlist_align_1shot'].append('-importGroundControlPoints ' + str(config['constantgcps']) + ' ' + config['gcpsparamfile'])
                    if not Path(config['constantgcps']).is_file() and Path(config['gcpsparamfile']).is_file() and pseudoscan['gcpsfile']:
                        pseudoscan['commandlist_align_1shot'].append('-importGroundControlPoints ' + str(pseudoscan['gcpsfile'][0]) + ' ' + config['gcpsparamfile'] )
                
                pseudoscan['commandlist_align_1shot'].append('-align')
                pseudoscan['commandlist_align_1shot'].append('-selectMaximalComponent')
                pseudoscan['commandlist_align_1shot'].append('-selectAllImages')
                #pseudoscan['commandlist_align_1shot'].append('-lockPoseForContinue true')
                pseudoscan['commandlist_align_1shot'].append('-exportRegistration ' + str(pseudoscan['RCoutputfolder']) + '/shot1_camparam.csv ' + config['paramexportconfig'])
                pseudoscan['commandlist_align_1shot'].append('-save')
                pseudoscan['commandlist_align_1shot'].append('-quit')
                pseudoscan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_align_1shot', rccmdpathfield='align_1shot-rccmdpath')
                RClib.executeRCCMDuseRCproject(pseudoscan, rccmdpathfield='align_1shot-rccmdpath', headless=False )
                RClib.writeProcessingstateFile(pseudoscan)
            print(len(scandf2a.iloc[0]['imagedf']))

        scandf3 = scandf3.append(pseudoscan)
else:
    scandf3 = scandf2a

In [ ]:
%load_ext autoreload
%autoreload 2
config = RClib.loadconfigs('.\config_sedimentcores.json')
scandf_calibparams = pd.DataFrame()
scandf_precalib = scandf2a
if config['calibratecameraparameters']:
    for index, scan in scandf_precalib.iterrows():
        print('Number of images before import: ' + str(len(scan['imagedf'])))
        scan2 = RClib.importRegisteredParameters(scan)
        print('Number of images: ' + str(len(scan2['imagedf'])))
        print('Number of calibrations: ' + str(len(scan2['imagedf'][scan2['imagedf']['f'].notnull()])))
        scandf_calibparams = scandf_calibparams.append(scan2)
    resultparams = RClib.plotParametersStats(scandf_calibparams)
    # save resultparams to json using pandas
    pd.DataFrame(resultparams).to_json(str(Path(config['workspace']) / 'fixCamparams.json'), orient='records')






In [ ]:
%load_ext autoreload
%autoreload 2
config = RClib.loadconfigs('.\config_sedimentcores.json')

        
scandf4=pd.DataFrame()
if config['align']:
    for index,scan in scandf3.iterrows():

        if not scan['imagedf'] is None :
            if not config['align_1shot']: 
                scan = RClib.createRCproject(scan)
            if config['align_1shot']:
                print('align_1shot has been done so the imagedf needs to be set back ', len(scandf2a[scandf2a['id']==scan['id']].iloc[0]['imagedf']))
                scan['imagedf'] = scandfBefore1shot[scandfBefore1shot['id']==scan['id']].iloc[0]['imagedf']

            scan = RClib.makeImagelist(scan, imagelistname=scan['id'])
            if config['maskobject_QS3D']:  
                scan = RClib.makeImagelist(scan, imagelistname= scan['id'] +'_masks' , imagefield='maskimg_path')
            #scan = RClib.createRCproject(scan)
            scan =RClib.readProcessingstateFile(scan)    
            if not RClib.checkProcessingstate(scan,command='commandlist_alignsettings'):
                scan['commandlist_alignsettings']=Path(config['alignmentsettings'])
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_alignsettings' )
            if not RClib.checkProcessingstate(scan,command='align-rccmdpath'):
                scan['commandlist_align'] = []
                scan['commandlist_align'].append('-add ' + str(scan['list_dev-img_path']))
                if config['maskobject_QS3D']:
                    scan['commandlist_align'].append('-add ' + str(scan['list_maskimg_path']))
                if 'imageparams' in config and config['imageparams'] == 'constant':
                    scan['commandlist_align'].append('-selectAllImages')
                    scan['commandlist_align'].append('-setFeatureSource 2')
                    scan['commandlist_align'].append('-setPriorLensGroup 1')
                    scan['commandlist_align'].append('-setPriorCalibrationGroup 1')
                if 'imageparams' in config and config['imageparams'] == 'variable':
                    scan['commandlist_align'].append('-selectAllImages')
                    scan['commandlist_align'].append('-setFeatureSource 2')
                    scan['commandlist_align'].append('-setPriorLensGroup -1')
                    scan['commandlist_align'].append('-setPriorCalibrationGroup -1')
                if 'imageparams' in config and config['imageparams'] == 'fixed' and Path(config['imageparamsfile']).is_file():
                    #read imageparams from json-file which is defined in config
                    imageparams = pd.read_json(config['imageparamsfile'], orient='records')
                    #makeImagelist of each group of images with same 'cam_id'
                    # and add to commandlist_align
                    for name, group in scan['imagedf'].groupby('cam_id'):
                        pseudoscan = scan.copy()
                        pseudoscan['imagedf'] = group
                        group = RClib.makeImagelist(pseudoscan, imagelistname=scan['id'] + '_' + str(name) + '_images' , imagefield='dev-img_path')
                        scan['commandlist_align'].append('-importImageSelection ' + str(group['list_dev-img_path']))
                        scan['commandlist_align'].append('-editInputSelection "inpCalibration=2"')
                        scan['commandlist_align'].append('-editInputSelection "inpFocal=' + str(imageparams[imageparams['cam_id']== name].iloc[0]['f_mean'])+ '"')
                        scan['commandlist_align'].append('-editInputSelection "inpPPX=' + str(imageparams[imageparams['cam_id']== name].iloc[0]['px_mean'])+ '"')
                        scan['commandlist_align'].append('-editInputSelection "inpPPY=' + str(imageparams[imageparams['cam_id']== name].iloc[0]['py_mean'])+ '"')
                        scan['commandlist_align'].append('-editInputSelection "inpDistortion=2"') 
                        scan['commandlist_align'].append('-editInputSelection "inpDistortionModel=1"')
                        scan['commandlist_align'].append('-editInputSelection "inpRadial1=-0.049831"')
                if config['use_aligngroups']:
                    for name, shotgroup in scan['imagedf'].groupby(['roundnumber','shotnumber']):
                        print(len(shotgroup) )
                        shotscan = scan.copy()
                        shotscan['imagedf'] = shotgroup
                        print(len(shotscan['imagedf']))
                        shotscan = RClib.makeImagelist(shotscan, imagelistname=str(name[0]) + '_' + str(name[1])+ '_images' , imagefield='dev-img_path')
                        scan['commandlist_align'].append('-selectAllImages')
                        scan['commandlist_align'].append('-setFeatureSource 2')
                        scan['commandlist_align'].append('-enableAlignment true')
                        scan['commandlist_align'].append('-importImageSelection ' + str(shotscan['list_dev-img_path']))
                        scan['commandlist_align'].append('-invertImageSelection')
                        scan['commandlist_align'].append('-enableAlignment false')
                        scan['commandlist_align'].append('-align')
                        scan['commandlist_align'].append('-renameSelectedComponent ' + str(name[0]) + '_' + str(name[1]))
                
                    scan['commandlist_align'].append('-selectAllImages')
                    scan['commandlist_align'].append('-setFeatureSource 1')
                    scan['commandlist_align'].append('-enableAlignment true')
                    scan['commandlist_align'].append('-align')

                if config['use_gcps']:
                    scan['commandlist_align'].append('-detectMarkers')
                    if Path(config['constantgcps']).is_file() and Path(config['gcpsparamfile']).is_file():
                        scan['commandlist_align'].append('-importGroundControlPoints ' + str(config['constantgcps']) + ' ' + config['gcpsparamfile'])
                    if not Path(config['constantgcps']).is_file() and Path(config['gcpsparamfile']).is_file() and scan['gcpsfile']:
                        scan['commandlist_align'].append('-importGroundControlPoints ' + str(scan['gcpsfile'][0]) + ' ' + config['gcpsparamfile'] )
                if not config['use_aligngroups']:
                    scan['commandlist_align'].append('-align')
                if 'sedimentcores_selectstraight' in config and config['sedimentcores_selectstraight']:
                    scan['camregistrationpath'] = [Path(scan['RCoutputfolder']) / "camregistration.csv"]

                    scan['commandlist_align'].append('-selectAllImages')
                    if not scan['camregistrationpath'][0].is_file():
                        print('camregistrationpath is not defined')

                        scan['commandlist_align'].append('-exportRegistration ' + str(scan['camregistrationpath'][0]) + ' ' + config['cameraparamsconfig'])
                scan['commandlist_align'].append('-save')
                scan['commandlist_align'].append('-quit')
                scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_align', rccmdpathfield='align-rccmdpath')
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='align-rccmdpath', headless=False )
                RClib.writeProcessingstateFile(scan)
            scandf4 = scandf4.append(scan)
else:
    scandf4 = scandf3
#pd.write_csv(Path(config['workspace']) / 'rcprocessingdf.csv', sep=';')
#scandf2.to_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))

In [ ]:
%load_ext autoreload
%autoreload 2
config = RClib.loadconfigs('.\config_sedimentcores.json')
        
scandf5=pd.DataFrame()
if config['scale_RS3D']:
    for index,scan in scandf4.iterrows():
        if not scan['imagedf'] is None :
            pseudoscan = scan
            print(scan['imagedf']['roundnumber'])
            imagedf_only1shot = scan['imagedf'][scan['imagedf']['shotnumber']==1]
            pseudoscan['imagedf']  = imagedf_only1shot[imagedf_only1shot['roundnumber']=='round0']
            pseudoscan = RClib.makeImagelist(pseudoscan, imagelistname=scan['id'] +'_1shot')
            #scan = RClib.makeImagelist(scan, imagelistname= scan['id'] +'_masks' , imagefield='maskimg_path')
            scan =RClib.readProcessingstateFile(scan)    
            if not RClib.checkProcessingstate(scan,command='scale_RS3D-rccmdpath'):
                scan['commandlist_scale_RS3D'] = []
                scan['commandlist_scale_RS3D'].append('-importImageSelection ' + str(pseudoscan['list_dev-img_path']))
                scan['commandlist_scale_RS3D'].append('-detectMarkers')
                if Path(config['constantgcps']).is_file() and Path(config['gcpsparamfile']).is_file():
                    scan['commandlist_scale_RS3D'].append('-importGroundControlPoints ' + str(config['constantgcps']) + ' ' + config['gcpsparamfile'])
                scan['commandlist_scale_RS3D'].append('-align')
                scan['commandlist_scale_RS3D'].append('-save')
                scan['commandlist_scale_RS3D'].append('-quit')
                scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_scale_RS3D', rccmdpathfield='scale_RS3D-rccmdpath')
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='scale_RS3D-rccmdpath', headless=False )
                RClib.writeProcessingstateFile(scan)
            scandf5 = scandf5.append(scan)
else:
    scandf5 = scandf4

In [ ]:
%load_ext autoreload
%autoreload 2
config = RClib.loadconfigs('.\config_sedimentcores.json')
scandf6=pd.DataFrame()
if 'sedimentcores_selectstraight' in config and config['sedimentcores_selectstraight']:
    for index,scan in scandf4.iterrows():
        print(scan['imagedf'].columns)
        scan['imagedf']['filename'] = scan['imagedf']['dev-img_path'].apply(lambda x: Path(x).name)
        scan = RClib.loadCameraPoses(scan)
        scan = RClib.get_frontal_cameras(scan)
        scan = RClib.makeImagelist(scan, imagelistname=scan['id'] + '_frontal')
        scandf6 = scandf6.append(scan)
else:
    scandf6 = scandf5

In [ ]:
%load_ext autoreload
%autoreload 2
config = RClib.loadconfigs('.\config_sedimentcores.json')
        
scandf7=pd.DataFrame()
if 'sedimentcores_selectstraight' in config and config['sedimentcores_selectstraight']:
    for index,scan in scandf6.iterrows():
        if not scan['imagedf'] is None :
            scan =RClib.readProcessingstateFile(scan)    
            if not RClib.checkProcessingstate(scan,command='frontal-align-rccmdpath'):
                scan['commandlist_frontal-align'] = []
                scan['commandlist_frontal-align'].append('-importImageSelection ' + str(scan['list_dev-img_path']))
                scan['commandlist_frontal-align'].append('-invertImageSelection')
                scan['commandlist_frontal-align'].append('-enableAlignment false')
                scan['commandlist_frontal-align'].append('-align')
                scan['commandlist_frontal-align'].append('-renameSelectedComponent ' + str(scan['id']) + '_frontal')
                scan['commandlist_frontal-align'].append('-save')
                scan['commandlist_frontal-align'].append('-quit')
                scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_frontal-align', rccmdpathfield='frontal-align-rccmdpath')
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='frontal-align-rccmdpath', headless=False )
                RClib.writeProcessingstateFile(scan)
            scandf7 = scandf7.append(scan)
else:
    scandf7 = scandf6

In [ ]:
%load_ext autoreload
%autoreload 2
scandf7= scandf7.apply(RClib.defineResultOutput, foldername='04_GEO', axis=1)
#while 'reconstruct-rccmdpath'.isin(scandf3['processingstate'])] and 'model-rccmdpath'.notin(scandf3['processingstate']) :
    #tobemodeleddf = scandf3['reconstruct-rccmdpath'.isin(scandf3['processingstate']) and 'model-rccmdpath'.notin(scandf3['processingstate']) ]
scandf8 = pd.DataFrame()
config = RClib.loadconfigs('.\config_sedimentcores.json')
if config['quick_mosaic']:
    for index,scan in scandf7.iterrows():
        if 'constantrcbox' in config.keys() and Path(config['constantrcbox']).is_file():
            scan['rcboxpath'] = Path(config['constantrcbox'])
        else:
            scan['rcboxpath'] =  Path(scan['scan_dir']) / "base.rcbox"
        scan =RClib.readProcessingstateFile(scan)    
        if RClib.checkProcessingstate(scan,command='align-rccmdpath') and scan['rcboxpath'].is_file():
            if not RClib.checkProcessingstate(scan,command='commandlist_quickreconstructsettings'):
                scan['commandlist_quickreconstructsettings']=Path(config['reconstructsettings'])
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_quickreconstructsettings')
            if not RClib.checkProcessingstate(scan,command='quick_mosaic-rccmdpath'): 
                scan['commandlist_quick_mosaic'] = []
                if 'sedimentcores_selectstraight' in config and config['sedimentcores_selectstraight']:
                    scan['commandlist_quick_mosaic'].append('-selectComponent ' + scan['id'] + '_frontal')
                scan['commandlist_quick_mosaic'].append('-setReconstructionRegion ' + str(scan['rcboxpath'])) 
                print ('-setReconstructionRegion ' + str(scan['rcboxpath']))
                scan['commandlist_quick_mosaic'].append('-calculatePreviewModel') 
                scan['commandlist_quick_mosaic'].append('-renameSelectedModel ' + scan['id'] + '_preview')
                rcorthotree = RClib.generate_rcortho(scan['rcbox'], 2500, 31250, "general mosaicing", scan['id'] + '_preview', 13)
                scan['rcorthoboxpath'] = Path(scan['RCoutputfolder']) / (scan['id'] + '_preview.rcortho')
                print(rcorthotree)
                ET.ElementTree(rcorthotree).write(scan['rcorthoboxpath'])
                scan['commandlist_quick_mosaic'].append('-calculateOrthoProjection ' + str(scan['rcorthoboxpath']))
                scan['commandlist_quick_mosaic'].append('-exportOrthoProjection ' + str(scan['Resultoutputfolder']) + '/' + str(scan['id'] + '_ortho_preview.jpeg') + ' ' + str(config['quick_mosaic_exportsettings']))
                 
                scan['commandlist_quick_mosaic'].append('-save')
                scan['commandlist_quick_mosaic'].append('-quit')  
                scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_quick_mosaic', rccmdpathfield='quick_mosaic-rccmdpath')
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='quick_mosaic-rccmdpath' )
                RClib.writeProcessingstateFile(scan)
        scandf8 = scandf8.append(scan)
else:
    scandf8 = scandf7

In [ ]:
%load_ext autoreload
%autoreload 2
#scandf4= scandf4.apply(RClib.defineResultOutput, foldername='04_GEO', axis=1)
#while 'reconstruct-rccmdpath'.isin(scandf3['processingstate'])] and 'model-rccmdpath'.notin(scandf3['processingstate']) :
    #tobemodeleddf = scandf3['reconstruct-rccmdpath'.isin(scandf3['processingstate']) and 'model-rccmdpath'.notin(scandf3['processingstate']) ]
scandf9 = pd.DataFrame()
if config['reconstruct']:
    for index,scan in scandf8.iterrows():
        if 'constantrcbox' in config.keys() and Path(config['constantrcbox']).is_file():
            scan['rcboxpath'] = Path(config['constantrcbox'])
        else:
            scan['rcboxpath'] =  Path(scan['scan_dir']) / "base.rcbox"
        scan =RClib.readProcessingstateFile(scan)    
        if RClib.checkProcessingstate(scan,command='align-rccmdpath') and scan['rcboxpath'].is_file():
            if not RClib.checkProcessingstate(scan,command='commandlist_reconstructsettings'):
                scan['commandlist_reconstructsettings']=Path(config['reconstructsettings'])
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_reconstructsettings')
            if not RClib.checkProcessingstate(scan,command='reconstruct-rccmdpath'): 
                scan['commandlist_reconstruct'] = []
                if 'sedimentcores_selectstraight' in config and config['sedimentcores_selectstraight']:
                    scan['commandlist_reconstruct'].append('-selectComponent ' + str(scan['id']) + '_frontal')
                scan['commandlist_reconstruct'].append('-setReconstructionRegion ' + str(scan['rcboxpath'])) 
                print ('-setReconstructionRegion ' + str(scan['rcboxpath']))
                scan['commandlist_reconstruct'].append('-calculateHighModel')  
                scan['commandlist_reconstruct'].append('-renameSelectedModel ' + scan['id'] + '_highpoly')    
                scan['commandlist_reconstruct'].append('-save')
                scan['commandlist_reconstruct'].append('-quit')  
                scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_reconstruct', rccmdpathfield='reconstruct-rccmdpath')
                print('Starting reconstruction for  ', scan['id'])
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='reconstruct-rccmdpath' )
                RClib.writeProcessingstateFile(scan)
        scandf9 = scandf9.append(scan)
else:
    scandf9 = scandf8

In [ ]:
scandf10 = pd.DataFrame()
if config['texture']:
    for index,scan in scandf9.iterrows():
        scan = RClib.readProcessingstateFile(scan)
        if RClib.checkProcessingstate(scan,command='reconstruct-rccmdpath'):
            if not RClib.checkProcessingstate(scan,command='commandlist_texturingsettings'):
                scan['commandlist_texturingsettings']=Path(config['texturingsettings'])
                print('Texturing settings set for ', scan['id'])
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_texturingsettings')
            if not RClib.checkProcessingstate(scan,command='texturing-rccmdpath'):
                scan['commandlist_texturing'] = []
                if 'sedimentcores_selectstraight' in config and config['sedimentcores_selectstraight']:
                    scan['commandlist_texturing'].append('-selectComponent ' + str(scan['id']) + '_frontal')
                scan['commandlist_texturing'].append('-selectModel ' + scan['id'] + '_highpoly')
                #scan['commandlist_texturing'].append('-selectLargestModelComponent')
                #scan['commandlist_texturing'].append('-invertTrianglesSelection')
                scan['commandlist_texturing'].append('-smooth')
                scan['commandlist_texturing'].append('-correctColors geometry') 
                scan['commandlist_texturing'].append('-unwrap') 
                scan['commandlist_texturing'].append('-calculateTexture')
                scan['commandlist_texturing'].append('-simplify '+ str(config['simplify_settings']))
                scan['commandlist_texturing'].append('-renameSelectedModel ' + scan['id'] + '_100k')
                scan['modelspaths'] = {'modelpath_100k': scan['Resultoutputfolder'] / Path(config['exportname_prefix'] + scan['id'] + '_100k.obj')}
                print(scan['modelspaths'])
                scan['commandlist_texturing'].append('-exportModel ' + scan['id'] + '_100k ' + str(scan['modelspaths'].get('modelpath_100k')) + ' ' + config['exportsettings1'] )
                scan['commandlist_texturing'].append('-save')
                scan['commandlist_texturing'].append('-quit')  
                scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_texturing', rccmdpathfield='texturing-rccmdpath')
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='texturing-rccmdpath' )
                RClib.writeProcessingstateFile(scan)
        scandf10 = scandf10.append(scan)
else:
    scandf10=scandf9


In [ ]:
if config['orthomosaic']:
    for index,scan in scandf3.iterrows():
        scan = RClib.readProcessingstateFile(scan)
        if RClib.checkProcessingstate(scan,command='reconstruct-rccmdpath'):
            if not RClib.checkProcessingstate(scan,command='commandlist_orthoimage'):
                if 'constantorthobox' in config.keys() and Path(config['constantorthobox']).is_file():
                    scan['orthoboxpath'] = Path(config['constantorthobox'])
                    scan['commandlist_orthoimage'] = []
                    scan['commandlist_orthoimage'].append('-selectModel ' + scan['id'] + '_highpoly') 
                    scan['commandlist_orthoimage'].append('-calculateOrthoProjection ' + scan['orthoboxpath'] ) 
                    scan['commandlist_orthoimage'].append('-exportOrthoProjection ' + scan['orthoboxpath'] ) 
            

In [ ]:
%load_ext autoreload
%autoreload 2
if config['orthobox2gis']:
    for index,scan in scandf.iterrows():
        #print(scan.keys())
        RClib.read_rcorthobox(scan)
        #print(scan['orthobox'])
        scan['orthobox'].to_file(config['workspace'] + '/Orthoboxes_utm38.shp', driver='ESRI Shapefile')